We are extending the 2D flocking model to three dimensions and testing how alignment strength, cohesion, and noise affect the flock shape and density. The main question is whether birds form compact spherical groups or elongated streams when we add 3D movement.

The approach uses PCA to measure the flock shape by finding its three principal axes, like length, width, and thickness. We also track density to see if flocks spread out or stay tight under different conditions. Most importantly, we ignore the first 60 percent of each simulation as burn in time since the flock needs time to organize itself from random initial positions.

In [ ]:
import sys
import os
import time
from pathlib import Path
import numpy as np
from src.flocking_sim_3d import run_simulation
from src.geometry import unwrap_periodic, pca_axes, flock_volume_from_pca, hull_volume

PROJECT_ROOT = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

print("CWD:", Path.cwd())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("src import test:", (PROJECT_ROOT / "src").exists())

First we need helper functions to handle the periodic boundaries. Since the simulation wraps around at the edges, a flock might appear split across opposite sides of the box. We unwrap positions relative to the center of mass so PCA sees the flock as one continuous group rather than scattered fragments.

We calculate volume two ways. PCA assumes the flock is roughly ellipsoidal, which works well for cohesive groups. Convex hull wraps tightly around the actual positions, which is better for irregular shapes. Usually PCA is smoother and more stable for time series analysis.

The core analysis function takes simulation history and computes metrics only on the latter 40 percent of frames after burn in. For each frame we measure density, hull volume, and shape ratios. If both ratios are close to 1, the flock is spherical. If one is much smaller, the flock is disk like or stream like.

In [ ]:
def _standardize_history(history):
    h = np.asarray(history, dtype=float)
    if h.ndim != 3:
        raise ValueError(f"history must be 3D, got shape={h.shape}")
    if h.shape[-1] == 3:
        return h
    if h.shape[1] == 3:
        return np.transpose(h, (0, 2, 1))
    if h.shape[0] == 3:
        return np.transpose(h, (1, 2, 0))
    raise ValueError(f"cannot infer axis for xyz=3, got shape={h.shape}")


def compute_metrics(history, box_size, burn_frac=0.6):
    H = _standardize_history(history)
    T = H.shape[0]
    start = int(T * burn_frac)
    frames = H[start:] if start < T else H[-1:]

    dens = []
    hullV = []
    I2I1 = []
    I3I1 = []

    for pos in frames:
        pos_u = unwrap_periodic(pos, box_size)
        Vpca, (I1, I2, I3) = flock_volume_from_pca(pos_u)
        dens.append(pos_u.shape[0] / (Vpca + 1e-12))
        Vh = hull_volume(pos_u)
        hullV.append(Vh)
        I2I1.append(I2 / (I1 + 1e-12))
        I3I1.append(I3 / (I1 + 1e-12))

    dens = np.asarray(dens, dtype=float)
    hullV = np.asarray(hullV, dtype=float)
    I2I1 = np.asarray(I2I1, dtype=float)
    I3I1 = np.asarray(I3I1, dtype=float)

    return {
        "density_mean": float(np.mean(dens)),
        "density_std": float(np.std(dens)),
        "hullV_mean": float(np.mean(hullV)),
        "hullV_std": float(np.std(hullV)),
        "I2I1_mean": float(np.mean(I2I1)),
        "I2I1_std": float(np.std(I2I1)),
        "I3I1_mean": float(np.mean(I3I1)),
        "I3I1_std": float(np.std(I3I1)),
    }

Now we define the parameter sweep. We vary alignment strength, cohesion, and noise. For each combination we run multiple seeds and average the results. The burn fraction of 0.6 means we throw away the first 60 percent of frames.

In [ ]:
def run_one_setting(params, seed):
    kwargs = dict(params)
    kwargs["seed"] = int(seed)
    history = run_simulation(**kwargs)
    return history


def sweep_3d(
    align_vals,
    cohesion_vals,
    noise_vals,
    seeds,
    sim_params,
    burn_frac=0.6,
    out_path="sweep3d_align_cohesion_noise.npz",
    overwrite=True,
    progress_every=1,
):
    align_vals = np.asarray(align_vals, dtype=float)
    cohesion_vals = np.asarray(cohesion_vals, dtype=float)
    noise_vals = np.asarray(noise_vals, dtype=float)
    seeds = list(seeds)

    if (not overwrite) and os.path.exists(out_path):
        raise FileExistsError(out_path)

    shape = (len(align_vals), len(cohesion_vals), len(noise_vals))

    density = np.full(shape, np.nan)
    density_std_across_seeds = np.full(shape, np.nan)
    hullV = np.full(shape, np.nan)
    hullV_std_across_seeds = np.full(shape, np.nan)
    I2I1 = np.full(shape, np.nan)
    I2I1_std_across_seeds = np.full(shape, np.nan)
    I3I1 = np.full(shape, np.nan)
    I3I1_std_across_seeds = np.full(shape, np.nan)

    total = len(align_vals) * len(cohesion_vals) * len(noise_vals) * len(seeds)
    done = 0
    t0 = time.time()

    for i, align in enumerate(align_vals):
        for j, cohesion in enumerate(cohesion_vals):
            for k, noise in enumerate(noise_vals):
                runs = []
                for seed in seeds:
                    params = dict(sim_params)
                    params["align"] = float(align)
                    params["cohesion"] = float(cohesion)
                    params["noise"] = float(noise)

                    history = run_one_setting(params, seed)
                    m = compute_metrics(history, box_size=float(sim_params["box_size"]), burn_frac=burn_frac)
                    runs.append(m)

                    done += 1
                    if progress_every and (done % progress_every == 0):
                        elapsed = time.time() - t0
                        rate = done / max(elapsed, 1e-9)
                        print(f"[{done}/{total}] align={align:.3f} cohesion={cohesion:.3f} noise={noise:.3f} seed={seed} | {rate:.2f} runs/s")

                dens_means = np.array([m["density_mean"] for m in runs], dtype=float)
                hull_means = np.array([m["hullV_mean"] for m in runs], dtype=float)
                i2i1_means = np.array([m["I2I1_mean"] for m in runs], dtype=float)
                i3i1_means = np.array([m["I3I1_mean"] for m in runs], dtype=float)

                density[i, j, k] = float(np.mean(dens_means))
                density_std_across_seeds[i, j, k] = float(np.std(dens_means))
                hullV[i, j, k] = float(np.mean(hull_means))
                hullV_std_across_seeds[i, j, k] = float(np.std(hull_means))
                I2I1[i, j, k] = float(np.mean(i2i1_means))
                I2I1_std_across_seeds[i, j, k] = float(np.std(i2i1_means))
                I3I1[i, j, k] = float(np.mean(i3i1_means))
                I3I1_std_across_seeds[i, j, k] = float(np.std(i3i1_means))

    np.savez(
        out_path,
        align_vals=align_vals,
        cohesion_vals=cohesion_vals,
        noise_vals=noise_vals,
        seeds=np.array(seeds, dtype=int),
        burn_frac=float(burn_frac),
        density=density,
        density_std_across_seeds=density_std_across_seeds,
        hullV=hullV,
        hullV_std_across_seeds=hullV_std_across_seeds,
        I2I1=I2I1,
        I2I1_std_across_seeds=I2I1_std_across_seeds,
        I3I1=I3I1,
        I3I1_std_across_seeds=I3I1_std_across_seeds,
        **{f"sim_{k}": np.array(v) if isinstance(v, (list, tuple, np.ndarray)) else v for k, v in sim_params.items()},
    )

    return out_path

Running the sweep with a relatively coarse grid. We test 5 alignment values, 4 cohesion values, 5 noise values, and 3 random seeds per combination. That gives us 300 total simulations. Each run is 600 time steps but we only analyze the last 240 frames.

In [ ]:
align_vals = [0.0, 0.5, 1.0, 1.5, 2.0]
cohesion_vals = [0.0, 0.2, 0.4, 0.6]
noise_vals = [0.0, 0.03, 0.06, 0.09, 0.12]
seeds = [0, 1, 2]

sim_params = dict(
    N=200,
    steps=600,
    box_size=1.0,
    R=0.15,
    speed=0.03,
    repulsion_radius=0.05,
    repulsion_strength=1.0,
    dt=0.1,
    save_every=1,
    softening=1e-6,
    use_predator=False,
)

out_file = sweep_3d(
    align_vals=align_vals,
    cohesion_vals=cohesion_vals,
    noise_vals=noise_vals,
    seeds=seeds,
    sim_params=sim_params,
    burn_frac=0.6,
    out_path="sweep3d_align_cohesion_noise.npz",
    overwrite=True,
    progress_every=1,
)

print("saved:", out_file)

Sanity check to verify metrics look reasonable. We expect density somewhere between 50 and 500, aspect ratios between 0.3 and 0.9, and non zero hull volume.

In [ ]:
test_params = dict(sim_params)
test_params.update(align=1.0, cohesion=0.4, noise=0.06, seed=0)

history_test = run_simulation(**test_params)
metrics_test = compute_metrics(history_test, box_size=float(sim_params["box_size"]), burn_frac=0.6)

print("Sanity check metrics:")
for k, v in metrics_test.items():
    print(f"{k:>20s}: {v:.6f}")